<a href="https://colab.research.google.com/github/plaban1981/Stable_Diffusion_codes/blob/main/Generate_Kids'_Story_Illustrations_with_AI_in_Just_One_Click!.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Author: [Ramsri Goutham](https://twitter.com/ramsri_goutham)

In [ ]:
!pip install --quiet segmind==0.2.3

In [ ]:
!pip install --quiet gradio==3.43.2

In [ ]:
from getpass import getpass
openaikey = getpass('Enter the openai API key: ')

In [ ]:
segmindkey = getpass('Enter the segmind API key: ')

In [ ]:
chatgpt_url = "https://api.openai.com/v1/chat/completions"
chatgpt_headers = {
    "content-type": "application/json",
    "Authorization":"Bearer {}".format(openaikey)}

In [ ]:
import requests
import json
from pprint import pprint

def fetch_illustrator_prompts(prompt, url, headers, max_retries=3):
    retries = 0

    while retries < max_retries:
        # Define the payload for the chat model
        messages = [
            {"role": "system", "content": "You are an expert author who can read children's stories and create short briefs for an illustrator, providing specific instructions, ideas, or guidelines for the illustrations you want them to create."},
            {"role": "user", "content": prompt}
        ]

        chatgpt_payload = {
            "model": "gpt-3.5-turbo-16k",
            "messages": messages,
            "temperature": 1.3,
            "max_tokens": 2000,
            "top_p": 1,
            "stop": ["###"]
        }

        # Make the request to OpenAI's API
        response = requests.post(url, json=chatgpt_payload, headers=headers)
        response_json = response.json()

        try:
            # Extract data from the API's response
            output = json.loads(response_json['choices'][0]['message']['content'].strip())
            # pprint(output)
            return output
        except json.JSONDecodeError as e:
            print(f"JSON decode error: {e}")
            retries += 1
            print(f"Retry attempt {retries} out of {max_retries}")

    # If max retries are reached without a successful response, return None or handle it as needed.
    return None


In [ ]:
import json

def generate_prompt(text, count):
    prompt_prefix = """{}
------------------------------------
Generate {} short briefs as a list from the above story to give as input to an illustrator to generate relevant children's story illustrations.
Strictly add no common prefix to briefs. Strictly generate each brief as a single sentence that contains all the necessary information.
Strictly output your response in a JSON list format, adhering to the following sample structure:""".format(json.dumps(text), json.dumps(count))

    sample_output = {"illustrations": ["...", "...", "..."]}

    prompt_postinstruction = "\nOutput:"

    return prompt_prefix + json.dumps(sample_output) + prompt_postinstruction



In [ ]:
# https://momlovesbest.com/short-moral-stories-kids

text = """Once, there was a boy who became bored when he watched over the village sheep grazing on the hillside. To entertain himself, he sang out, “Wolf! Wolf! The wolf is chasing the sheep!”
When the villagers heard the cry, they ran up the hill to drive the wolf away. But when they arrived, they saw no wolf. The boy was amused when he saw their angry faces.
“Don’t scream wolf when there is no wolf, boy!” the villagers warned. They angrily went back down the hill.
Later, the shepherd boy cried out once again, “Wolf! Wolf! The wolf is chasing the sheep!” To his amusement, the villagers came running up the hill to scare the wolf away.
As they saw there was no wolf, they said strictly, “Save your frightened cry for when there really is a wolf! Don’t cry ‘wolf’ when there is no wolf!” But the boy grinned at their words while they walked, grumbling down the hill once more.
Later, the boy saw a real wolf sneaking around his flock. Alarmed, he jumped on his feet and cried out as loud as he could, “Wolf! Wolf!” But the villagers thought he was fooling them again, and they didn’t come to help.
At sunset, the villagers went looking for the boy who hadn’t returned with their sheep. When they went up the hill, they found him weeping.
“There was a wolf here! The flock is gone! I cried out, ‘Wolf!’ but you didn’t come,” he wailed."""

count = 3
prompt = generate_prompt(text, count)

print (prompt)

In [ ]:
illustrator_prompts = fetch_illustrator_prompts(prompt,chatgpt_url,chatgpt_headers)
print (illustrator_prompts)
for each in illustrator_prompts['illustrations']:
  print (each)

In [ ]:
from segmind import SDXL
model = SDXL(segmindkey)

In [ ]:
import os
import io
import requests
from PIL import Image
import random

def generate_images(prompts, style):
    all_images =[]

    num_images = len(prompts)

    currentseed = random.randint(1, 1000000)
    print ("seed ",currentseed)

    negative_prompt = "photorealistic, realistic, photograph, deformed, mutated, stock photo, 35mm film, deformed, glitch, low contrast, noisy"

    for i, prompt in enumerate(prompts):

        final_prompt = "{},{}".format(prompt.replace('.',","),style)
        img = model.generate(prompt = final_prompt,negative_prompt=negative_prompt,samples = 1,style = style,scheduler="UniPC",
                             seed =currentseed, num_inference_steps=30)


        print (f"Image {i + 1}/{num_images} is generated")
        # img will be a PIL image
        all_images.append(img)

    return all_images

In [ ]:
style = "watercolor"
# style = "kawaii"
all_images = generate_images(illustrator_prompts['illustrations'],style)

In [ ]:
!pip install --quiet ipyplot

In [ ]:
import ipyplot
ipyplot.plot_images(all_images,labels=illustrator_prompts['illustrations'],img_width = 300)

In [ ]:
import gradio as gr
import shutil
import os

# https://momlovesbest.com/short-moral-stories-kids

story_1= """Once, there was a boy who became bored when he watched over the village sheep grazing on the hillside. To entertain himself, he sang out, “Wolf! Wolf! The wolf is chasing the sheep!”
When the villagers heard the cry, they ran up the hill to drive the wolf away. But when they arrived, they saw no wolf. The boy was amused when he saw their angry faces.
“Don’t scream wolf when there is no wolf, boy!” the villagers warned. They angrily went back down the hill.
Later, the shepherd boy cried out once again, “Wolf! Wolf! The wolf is chasing the sheep!” To his amusement, the villagers came running up the hill to scare the wolf away.
As they saw there was no wolf, they said strictly, “Save your frightened cry for when there really is a wolf! Don’t cry ‘wolf’ when there is no wolf!” But the boy grinned at their words while they walked, grumbling down the hill once more.
Later, the boy saw a real wolf sneaking around his flock. Alarmed, he jumped on his feet and cried out as loud as he could, “Wolf! Wolf!” But the villagers thought he was fooling them again, and they didn’t come to help.
At sunset, the villagers went looking for the boy who hadn’t returned with their sheep. When they went up the hill, they found him weeping.
“There was a wolf here! The flock is gone! I cried out, ‘Wolf!’ but you didn’t come,” he wailed."""

def create_illustrations(textInp, styleInp):

    prompt = generate_prompt(textInp, 3)
    illustrator_prompts = fetch_illustrator_prompts(prompt, chatgpt_url, chatgpt_headers)
    pil_images = generate_images(illustrator_prompts['illustrations'], styleInp)

    return pil_images

topics = ["watercolor", "comic book", "kawaii", "line art"]

with gr.Blocks() as demo:
    gr.Markdown("# Kid's story illustrations generator")
    text_input = gr.Textbox(value=story_1, label="Enter Text", lines=4,max_lines=6)
    genre = gr.Dropdown(choices=topics)
    btn_create_illustrations = gr.Button('Generate Illustrations')
    gallery = gr.Gallery(label="images", columns=3)

    btn_create_illustrations.click(fn=create_illustrations, inputs=[text_input, genre], outputs=[gallery])

demo.launch(debug=True, enable_queue=True)